# Task + Query augmentation
https://github.com/RenkunNi/MetaAug/tree/79d1a6a457be37258df50a9194946caeb86845a2

In [1]:
import torch
import torch.nn as nn
import torchvision
from torchvision import datasets, transforms
from torchvision.models import resnet50
from torch.utils.data import Dataset, DataLoader
import torchvision.models as models
import torch.nn.functional as F
from tqdm.auto import tqdm,trange
from pathlib import Path
import pandas as pd
import torch.nn.utils.prune as prune
import numpy as np
import random
import learn2learn as l2l
import pickle
import matplotlib.pyplot as plt

/home/samuel/.conda/envs/meta/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
seed = 42
torch.manual_seed(seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(seed)
np.random.seed(seed)
random.seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
device = "cuda" if torch.cuda.is_available() else "cpu"

In [3]:
ways = 5 
shots =5

In [4]:
with open("./data/test.pkl",'rb') as f:
    test_data = pickle.load(f)
with open("./data/train.pkl",'rb') as f:
    train_data = pickle.load(f)
with open("./data/validation.pkl",'rb') as f:
    val_data = pickle.load(f)

In [5]:
train_data['images'].shape

(38400, 3, 84, 84)

In [6]:
val_data['images'].shape

(9600, 3, 84, 84)

## Task augmentation

In [7]:
%%time
images = train_data["images"]
labels = train_data["labels"]
num_labels = np.unique(labels).shape[0]
num_new_labels = num_labels*3
n_data = int(len(labels)/num_labels)
for i in range(num_labels):
    images_of_one_class = images[i*n_data:(i+1)*n_data] #600
    new_img1 = np.array([np.rot90(img.copy(), axes=(2,1)) for img in images_of_one_class]) 
    new_img2 = np.array([np.rot90(img.copy(),2, axes=(2,1)) for img in images_of_one_class])
    new_img3 = np.array([np.rot90(img.copy(),3, axes=(2,1)) for img in images_of_one_class])
    train_data["images"] = np.concatenate((train_data["images"],new_img1,new_img2,new_img3))
    train_data["labels"] = np.concatenate((train_data["labels"],[num_labels+3*i]*n_data,[num_labels+3*i+1]*n_data,[num_labels+3*i+2]*n_data))
    
    # print(train_data['labels'].shape)
    # break

CPU times: user 32 s, sys: 38.2 s, total: 1min 10s
Wall time: 1min 10s


In [8]:
train_data["images"].shape,train_data["labels"].shape

((153600, 3, 84, 84), (153600,))

In [11]:
%%time
images = val_data["images"]
labels = val_data["labels"]
num_labels = np.unique(labels).shape[0]
num_new_labels = num_labels*3
n_data = int(len(labels)/num_labels)
for i in range(num_labels):
    images_of_one_class = images[i*n_data:(i+1)*n_data]
    new_img1 = np.array([np.rot90(img.copy(), axes=(2,1)) for img in images_of_one_class]) 
    new_img2 = np.array([np.rot90(img.copy(),2, axes=(2,1)) for img in images_of_one_class])
    new_img3 = np.array([np.rot90(img.copy(),3, axes=(2,1)) for img in images_of_one_class])
    val_data["images"] = np.concatenate((val_data["images"],new_img1,new_img2,new_img3))
    val_data["labels"] = np.concatenate((val_data["labels"],[num_labels+3*i]*n_data,[num_labels+3*i+1]*n_data,[num_labels+3*i+2]*n_data))

CPU times: user 2.34 s, sys: 3.13 s, total: 5.47 s
Wall time: 5.47 s


In [12]:
val_data["images"].shape,val_data["labels"].shape

((38400, 3, 84, 84), (38400,))

## Build dataset

In [13]:
class Dataset(Dataset):
    def __init__(self,data,transform = None):
        self.transform = transform
        self.data=data
        self.images = np.transpose(self.data['images'],(0,2,3,1))
        self.labels = data['labels']
    def __len__(self): 
        return len(self.data['images'])
    def __getitem__(self,index):
        img = self.images[index]
        label = self.labels[index]
        if self.transform:
            img = self.transform(img)
        return img, label

In [14]:
train_transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.5,), (0.5,)),
        transforms.RandomHorizontalFlip(),
        # transforms.RandomAffine(degrees = 0, translate=(0.1,0.1)),
        # transforms.RandomRotation(20),#原本是10

    ])
val_transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.5,), (0.5,)),
        transforms.RandomHorizontalFlip(),
        # transforms.RandomRotation(20),#原本是10
        # transforms.RandomAffine(degrees = 0, translate=(0.1,0.1))
])

In [15]:
train_dataset = Dataset(train_data,train_transform)
val_dataset = Dataset(val_data, val_transform)

In [16]:
train_dataset = l2l.data.MetaDataset(train_dataset)
val_dataset = l2l.data.MetaDataset(val_dataset)

In [17]:
task_transforms = [
    l2l.data.transforms.FusedNWaysKShots(train_dataset, n=ways, k=2*shots),
    l2l.data.transforms.LoadData(train_dataset),
    l2l.data.transforms.RemapLabels(train_dataset),
    l2l.data.transforms.ConsecutiveLabels(train_dataset),
]
train_taskset =  l2l.data.TaskDataset(train_dataset, task_transforms, num_tasks=20000)
task_transforms = [
    l2l.data.transforms.FusedNWaysKShots(val_dataset, n = ways, k=2*shots),
    l2l.data.transforms.LoadData(val_dataset),
    l2l.data.transforms.RemapLabels(val_dataset),
    l2l.data.transforms.ConsecutiveLabels(val_dataset),
]
val_taskset =  l2l.data.TaskDataset(val_dataset, task_transforms, num_tasks=20000)

## Query augmentation

In [19]:
def rand_bbox(size, lam):
    W = size[2]
    H = size[3]
    cut_rat = np.sqrt(1. - lam)
    cut_w = int(W * cut_rat)
    cut_h = int(H * cut_rat)

    # uniform
    cx = np.random.randint(W)
    cy = np.random.randint(H)

    bbx1 = np.clip(cx - cut_w // 2, 0, W)
    bby1 = np.clip(cy - cut_h // 2, 0, H)
    bbx2 = np.clip(cx + cut_w // 2, 0, W)
    bby2 = np.clip(cy + cut_h // 2, 0, H)

    return bbx1, bby1, bbx2, bby2
def adaptation(task, learner, fas, loss_func, device, mode):
    data, labels = task
    data = data.to(device)
    labels = labels.to(device)
    sup_mask = np.array(([False]*5+[True]*5)*5)
    query_mask = ~sup_mask
    sup_data , query_data = data[sup_mask], data[query_mask]
    sup_labels , query_labels = labels[sup_mask], labels[query_mask]
    for step in range(fas): # inner loop
        pred = learner(sup_data)
        train_loss = loss_func(pred, sup_labels)
        learner.adapt(train_loss)
    r = np.random.rand(1)
    if  mode=="valid" or r > 0.5:
         # no query augmentation
        query_pred = learner(query_data)
        query_loss = loss_func(query_pred, query_labels)
    else:
        # generate mixed sample for query augmentation(CutMix)
        lam = np.random.beta(2., 2.)
        rand_index = torch.randperm(query_data.size()[0]).cuda()
        target_a = query_labels
        target_b = query_labels[rand_index]
        bbx1, bby1, bbx2, bby2 = rand_bbox(query_data.size(), lam)
        query_data[:, :, bbx1:bbx2, bby1:bby2] = query_data[rand_index, :, bbx1:bbx2, bby1:bby2]
        # adjust lambda to exactly match pixel ratio
        lam = 1 - ((bbx2 - bbx1) * (bby2 - bby1) / (query_data.size()[-1] * query_data.size()[-2]))
        # compute output
        query_pred = learner(query_data)
        query_loss = loss_func(query_pred, target_a) * lam + loss_func(query_pred, target_b) * (1. - lam)   
    query_acc = (torch.argmax(query_pred,1)==query_labels).sum()/len(query_labels)
    return query_loss, query_acc

In [ ]:
def train(EPOCHS, meta_model, train_taskset, val_taskset, loss_func, opt, device, model_name):
    log_file = f"./log/{model_name}.txt"
    res = {
        "train_acc_list" : [],
        "val_acc_list" : [],
        "train_loss_list" : [],
        "val_loss_list" : [],
    }
    last_val_loss=float("inf")
    for epoch in trange(EPOCHS):
        opt.zero_grad()
        train_loss = 0
        train_acc = 0
        val_loss = 0
        val_acc = 0
        for _ in range(meta_batch):
            learner = meta_model.clone(first_order = False)
            task = train_taskset.sample() # sample 一組 [50,3,84,84],[50]
            query_loss, query_acc = adaptation(task, learner,fas, loss_func, device,mode="train")
            query_loss.backward()
            train_loss += query_loss.item()
            train_acc += query_acc.item()
            #validation
            learner = meta_model.clone(first_order = False)
            task = val_taskset.sample() # sample 一組 [50,3,84,84],[50]
            query_loss, query_acc = adaptation(task, learner,fas, loss_func, device,mode="valid")
            val_loss+=query_loss.item()
            val_acc +=query_acc.item()
        res["train_acc_list"].append((train_acc/meta_batch))
        res["val_acc_list"].append((val_acc/meta_batch))
        res["train_loss_list"].append((train_loss/meta_batch))
        res["val_loss_list"].append((val_loss/meta_batch))
        for p in meta_model.parameters():
            p.grad.data.mul_(1.0 / meta_batch)
        opt.step()
        
        # write log file
        out = {
            "Epoch" : epoch,
            "train accuracy" : (train_acc/meta_batch),
            "validation accuracy" : (val_acc/meta_batch),
            "train loss" : (train_loss/meta_batch),
            "validation loss" : (val_loss/meta_batch)
        }
        with open(log_file,"a") as f:
            f.write(str(out) + '\n')
            
        # print log
        if (epoch+1)%100==0:
            print(f"Epoch {epoch+1} | Train loss :{train_loss/meta_batch} | Train accuracy : {train_acc/meta_batch}")
            print(f"Epoch {epoch+1} | Validation loss :{val_loss/meta_batch} | Validation accuracy : {val_acc/meta_batch}")
        # save model
        if (epoch+1)%500==0:
            MODEL_PATH=f"./model/{model_name}"
            Path(MODEL_PATH).mkdir(parents=True, exist_ok=True)
            torch.save(meta_model,Path(MODEL_PATH)/f"model_{epoch+1}.pt")
        # early stop
        n = 750
        if len(res["val_loss_list"])>n:
            if (epoch+1)%n == 0:
                avg_val_loss = sum(res["val_loss_list"][-n:])/n
                print("Average val_loss of last 500 epoch:",avg_val_loss)
                if last_val_loss < avg_val_loss:
                    print("Early stop!!!")
                    return res
                last_val_loss = avg_val_loss
    return res

In [20]:
def test(meta_model, test_data, loss_func, device):
    test_loss = 0
    test_acc = 0
    test_sup_images = test_data['sup_images']
    test_sup_labels = test_data['sup_labels']
    test_qry_images = test_data['qry_images']
    all_pred = []
    for i in trange(len(test_sup_images)):
        learner = meta_model.clone(first_order = False)
        sup_image = torch.tensor(test_sup_images[i]).to(device)
        sup_label = torch.tensor(test_sup_labels[i]).to(device)
        qry_image = torch.tensor(test_qry_images[i]).to(device)
        for step in range(fas): # inner loop
            pred = learner(sup_image)
            train_loss = loss_func(pred, sup_label)
            learner.adapt(train_loss)
        output_pred = torch.argmax(learner(qry_image),1).reshape(-1,1).detach().cpu().numpy()
        all_pred += [item for sublist in output_pred for item in sublist]
    return all_pred

In [ ]:
maml_lr = 0.5
lr = 1e-3
EPOCHS = 10000
meta_batch = 32
fas = 1
model = l2l.vision.models.ResNet12(output_size=ways).to(device)
meta_model = l2l.algorithms.MAML(model, lr=maml_lr,first_order=False)
opt = torch.optim.AdamW(meta_model.parameters(), lr=lr)
loss_func = nn.CrossEntropyLoss(reduction='mean')
results  = train(EPOCHS, meta_model, train_taskset, val_taskset, loss_func,opt, device, "resnet_task_augmentation_queryaug_v3")

  1%|▏                  | 100/10000 [31:29<51:53:24, 18.87s/it]

Epoch 100 | Train loss :1.659164372831583 | Train accuracy : 0.3149999922607094
Epoch 100 | Validation loss :1.578516859561205 | Validation accuracy : 0.30749999010004103


  2%|▎                | 200/10000 [1:02:56<51:37:59, 18.97s/it]

Epoch 200 | Train loss :1.4744772650301456 | Train accuracy : 0.36874999152496457
Epoch 200 | Validation loss :1.482232816517353 | Validation accuracy : 0.35499999043531716


  3%|▌                | 300/10000 [1:34:23<50:52:48, 18.88s/it]

...

 81%|████████████▉   | 8100/10000 [41:17:03<9:51:28, 18.68s/it]

Epoch 8100 | Train loss :0.6429456085897982 | Train accuracy : 0.7999999886378646
Epoch 8100 | Validation loss :0.7281881812959909 | Validation accuracy : 0.7224999833852053


 82%|█████████████   | 8200/10000 [41:48:13<9:20:07, 18.67s/it]

Epoch 8200 | Train loss :0.5745856761932373 | Train accuracy : 0.8524999804794788
Epoch 8200 | Validation loss :0.6356121366843581 | Validation accuracy : 0.7624999769032001


 82%|█████████████▏  | 8237/10000 [41:59:46<9:10:32, 18.74s/it]

In [1]:
def moving_average(x, w):
    return np.convolve(x, np.ones(w), 'valid') / w
df1 = pd.DataFrame(results["train_acc_list"],columns=['train_accuracy'])
df2 = pd.DataFrame([i for i in results["val_acc_list"]],columns=['val_accuracy'])
df = pd.concat([df1,df2],axis=1)
df.plot()

NameError: name 'pd' is not defined

In [ ]:
df1 = pd.DataFrame(results["train_loss_list"][1000:],columns=['train_loss'])
df2 = pd.DataFrame([i for i in results["val_loss_list"][1000:]],columns=['val_loss'])
df = pd.concat([df1,df2],axis=1)
df.plot()

# Testing

In [ ]:
torch.save(meta_model,"./model/resnet_task_augmentation_queryaug_v3/task_augmentation_v3.pt")

In [ ]:
model = torch.load("./model/resnet_task_augmentation_queryaug_v3/task_augmentation_v3.pt").to(device)
pred_res = test(model, test_data, loss_func, device)
df = pd.concat([pd.Series(range(15000)),pd.Series(pred_res)],axis = 1)
df = df.rename(columns={df.columns[0]:"Id",df.columns[1]:"Category"})
df.to_csv("submission_taskaug_v3.csv", index =False)

In [30]:
pred_res = test(meta_model, test_data, loss_func, device)
df = pd.concat([pd.Series(range(15000)),pd.Series(pred_res)],axis = 1)
df = df.rename(columns={df.columns[0]:"Id",df.columns[1]:"Category"})
df.to_csv("submission_taskaug.csv", index =False)

100%|████████████████████████| 600/600 [01:01<00:00,  9.70it/s]


In [289]:
for i in range(3):
    model_param=f"./model/resnet_task_augmentation/model_{8000+500*i}.pt"

    model = l2l.vision.models.ResNet12(output_size=ways).to(device)
    meta_model = l2l.algorithms.MAML(model, lr=maml_lr,first_order=False)
    param = torch.load(model_param)
    
    meta_model.load_state_dict(param)
    meta_model = meta_model.to(device)
    meta_model.eval()
    pred_res = test(meta_model, test_data, loss_func, device)
    df = pd.concat([pd.Series(range(15000)),pd.Series(pred_res)],axis = 1)
    df = df.rename(columns={df.columns[0]:"Id",df.columns[1]:"Category"})
    df.to_csv(f"submission_{7500+500*i}.csv", index =False)

100%|████████████████████████| 600/600 [00:27<00:00, 21.71it/s]


In [80]:
model_param=f"./model/resnet_task_augmentation/final_model.pt"

model = l2l.vision.models.ResNet12(output_size=ways).to(device)
meta_model = l2l.algorithms.MAML(model, lr=maml_lr,first_order=False)
param = torch.load(model_param)

meta_model.load_state_dict(param)
meta_model = meta_model.to(device)
meta_model.eval()
pred_res = test(meta_model, test_data, loss_func, device)
df = pd.concat([pd.Series(range(15000)),pd.Series(pred_res)],axis = 1)
df = df.rename(columns={df.columns[0]:"Id",df.columns[1]:"Category"})
df.to_csv(f"submission_final.csv", index =False)

NameError: name 'test' is not defined

In [83]:
meta_model.eval()
pred_res = test(meta_model, test_data, loss_func, device)
df = pd.concat([pd.Series(range(15000)),pd.Series(pred_res)],axis = 1)
df = df.rename(columns={df.columns[0]:"Id",df.columns[1]:"Category"})
df.to_csv("submission_final.csv", index =False)

100%|████████████████████████| 600/600 [00:27<00:00, 22.11it/s]
